In [1]:
import pandas as pd 
import requests
import json 
from requests.auth import HTTPBasicAuth
import base64
from configparser import ConfigParser

In [35]:
with open("../../../../hackathon-mvp/prisma/data/recipes_seed2.json", "r") as f:
    recipes = json.load(f)

In [36]:
recipes["recipes"][5]

{'title': 'Instant Pot Wild Rice Soup',
 'url': 'https://pinchofyum.com/wp-content/uploads/Instant-Pot-Wild-Rice-Soup-3.jpg',
 'ingredients': [{'amt': '8', 'unit': 'ounces', 'ingred': 'fresh mushrooms'},
  {'amt': '1', 'unit': 'cup', 'ingred': 'wild rice '},
  {'amt': '4', 'unit': 'cups', 'ingred': 'chicken broth'},
  {'amt': '1/2', 'unit': 'cup', 'ingred': 'flour'},
  {'amt': '1 1/2', 'unit': 'cups', 'ingred': 'milk '}],
 'instructions': ['Instant Pot soup base: Using the sauté function, melt the butter and sauté the onions and mushrooms until softened. Add the wild rice, the broth, the salt, and the spices; cook on high pressure for 35 minutes. Release the steam, add carrots, garlic, and celery, and cook on high pressure for another 10 minutes.Release steam using the valve on top.',
  'Make it creamy: You have two choices for making it creamy – A) Add 1 1/2 cups heavy cream to the soup (plus more broth as needed) and you’re done! This is slightly thinner / more brothy, but so rich an

In [37]:
API_ROOT = "https://api-ce.kroger.com/v1/"

In [38]:
conf = ConfigParser()
conf.read("/home/james/api_keys.conf")

['/home/james/api_keys.conf']

In [39]:
client =conf["KROGER"]["client_id"]
key = conf["KROGER"]["key"]

In [40]:
resp = requests.post("https://api-ce.kroger.com/v1/connect/oauth2/token",
        data= {"grant_type": "client_credentials",
            "scope":["product.compact"]
        },
        auth=HTTPBasicAuth(client, key)
)

In [42]:
token = json.loads(resp.text)["access_token"]

In [43]:
locations = requests.get("https://api-ce.kroger.com/v1/locations",
    params={"filter.zipCode.near":"23455"},
    headers={
        "Accept":"application/json",
        "Authorization":f"Bearer {token}"}
    )

In [44]:
locations.status_code

200

In [45]:
location_list = json.loads(locations.text)["data"]

In [46]:
store_num = location_list[0]['locationId']

In [52]:
groceries = []
for recipe_obj in recipes["recipes"]:
    for ingred in recipe_obj["ingredients"]:
        term = ingred["ingred"]
        products = requests.get("https://api-ce.kroger.com/v1/products",    
            params={"filter.locationId":str(store_num), "filter.term":term},
            headers={
                "Accept":"application/json",
                "Authorization":f"Bearer {token}"}
            )
        product_list = json.loads(products.text)["data"] if "data" in json.loads(products.text) else []
        for obj in product_list:
            groceries.append(
                {
                "name":obj["description"],
                "brand":None,
                "category":obj["categories"][0],
                "price": None if "price" not in obj["items"][0] else obj["items"][0]["price"][list(obj["items"][0]["price"].keys())[0]],
                "image_url":obj["images"][0]["sizes"][0]["url"] if "images" in obj else None,
                "upc":obj["upc"]
                }
            )
                

In [54]:
len(groceries)

1618

In [55]:
groceries

[{'name': 'Jovial Gluten Free Brown Rice Pasta Elbows',
  'brand': None,
  'category': 'Pasta, Sauces, Grain',
  'price': None,
  'image_url': 'https://www.kroger.com/product/images/xlarge/front/0081542101127',
  'upc': '0081542101127'},
 {'name': 'NOW Foods Livingnow® Organic Quinoa Macaroni & Rice Elbow Pasta',
  'brand': None,
  'category': 'Pasta, Sauces, Grain',
  'price': None,
  'image_url': 'https://www.kroger.com/product/images/xlarge/front/0073373906321',
  'upc': '0073373906321'},
 {'name': 'Ancient Harvest Organic Corn & Quinoa Elbow Pasta',
  'brand': None,
  'category': 'Pasta, Sauces, Grain',
  'price': None,
  'image_url': 'https://www.kroger.com/product/images/xlarge/left/0008912522000',
  'upc': '0008912522000'},
 {'name': 'Barilla Gluten Free Elbows Pasta',
  'brand': None,
  'category': 'Pasta, Sauces, Grain',
  'price': None,
  'image_url': 'https://www.kroger.com/product/images/xlarge/front/0007680800391',
  'upc': '0007680800391'},
 {'name': 'Tinkyada Elbow Brown

In [50]:
groceries2[-1]

{'name': 'Progresso Parmesan Bread Crumbs',
 'brand': None,
 'category': 'Baking Goods',
 'price': 2.99,
 'image_url': 'https://www.kroger.com/product/images/xlarge/front/0004119689121',
 'upc': '0004119689121'}

In [56]:
with open("../../../../hackathon-mvp/prisma/data/groceries_seed.json", "w") as f:
    json.dump(groceries, f)

In [187]:
groceries

[{'name': 'Juicy Juice 100% Apple Juice',
  'brand': None,
  'category': 'Beverages',
  'price': None,
  'image_url': 'https://www.kroger.com/product/images/xlarge/front/0088949727525',
  'upc': '0088949727525'},
 {'name': 'Smart Juice Organic Antioxidant Force Juice',
  'brand': None,
  'category': 'Natural & Organic',
  'price': None,
  'image_url': 'https://www.kroger.com/product/images/xlarge/front/0089435700258',
  'upc': '0089435700258'},
 {'name': 'Juicy Juice Orange Tangerine Juice',
  'brand': None,
  'category': 'Beverages',
  'price': None,
  'image_url': 'https://www.kroger.com/product/images/xlarge/front/0088949729505',
  'upc': '0088949729505'},
 {'name': 'Smart Juice Organic Black Mulberry Juice',
  'brand': None,
  'category': 'Natural & Organic',
  'price': None,
  'image_url': 'https://www.kroger.com/product/images/xlarge/front/0089435700256',
  'upc': '0089435700256'},
 {'name': 'Juicy Juice 100% Mango Juice Blend',
  'brand': None,
  'category': 'Beverages',
  'pric

In [186]:
obj

{'productId': '0081075701017',
 'upc': '0081075701017',
 'aisleLocations': [{'bayNumber': '3',
   'description': '5',
   'number': '5',
   'numberOfFacings': '1',
   'side': 'R',
   'shelfNumber': '6',
   'shelfPositionInBay': '3'},
  {'bayNumber': '3',
   'description': '5',
   'number': '5',
   'numberOfFacings': '1',
   'side': 'R',
   'shelfNumber': '6',
   'shelfPositionInBay': '3'},
  {'bayNumber': '3',
   'description': '5',
   'number': '5',
   'numberOfFacings': '1',
   'side': 'R',
   'shelfNumber': '6',
   'shelfPositionInBay': '3'},
  {'bayNumber': '3',
   'description': '5',
   'number': '5',
   'numberOfFacings': '1',
   'side': 'R',
   'shelfNumber': '6',
   'shelfPositionInBay': '3'}],
 'brand': 'Schar',
 'categories': ['Baking Goods'],
 'description': 'Schar Gluten Free Bread Crumbs',
 'items': [{'itemId': '0081075701017',
   'favorite': False,
   'fulfillment': {'curbside': False,
    'delivery': False,
    'inStore': False,
    'shipToHome': False},
   'size': '8.8 o